# Complete Micro-Learning Agent System with Gradio Interface

# 🛠️ Multi-Platform IT Implementation Guide Generator

**An AI-powered system that generates comprehensive, step-by-step pilot project implementation guides for any cloud platform with compliance requirements.**

---

## 📋 Overview

This system uses **three specialized AI agents** that work together to create production-ready implementation guides:

1. **🔧 Technical Expert Agent** - Searches platform-specific documentation and best practices
2. **⚖️ Compliance Specialist Agent** - Researches regulatory requirements and compliance frameworks  
3. **💰 Cost Optimization Agent** - Analyzes pricing and cost optimization strategies

### **Input:** Three specific requirements
- **Implementation Goal** - What you want to achieve (e.g., "10-year accounting records storage")
- **Technology Platform** - Which platform to use (e.g., "AWS S3", "Azure Blob Storage")
- **Compliance Requirements** - Which regulations to meet (e.g., "SOX compliance", "GDPR")

### **Output:** Complete implementation guide with:
- Executive summary and project scope
- 5-8 detailed implementation steps with code/configurations
- Compliance requirements and verification procedures
- Cost analysis and rollback plans
- Downloadable markdown documentation

---

## ⚙️ Setup Instructions

### **1. Prerequisites**
- Python 3.8+
- OpenAI API account and API key
- Jupyter Notebook environment

### **2. Install Required Packages**
```bash
pip install gradio agents pydantic python-dotenv
```

### **3. Environment Configuration**
Create a `.env` file in your project directory:
```env
OPENAI_API_KEY=your_openai_api_key_here
```

### **4. API Costs**
- **WebSearchTool**: ~$0.025 per search (3 searches per generation)
- **Rate limiting**: 5 requests per hour per user
- **Estimated cost**: ~$0.10-0.15 per complete guide generation

---

## 🎯 How to Use

### **Step 1: Run the System**
Execute all cells in this notebook. The system will launch a Gradio web interface with a shareable public URL.

### **Step 2: Generate Implementation Guides**
1. **Enter Implementation Goal**: Be specific (e.g., "Medical records storage with audit trails")
2. **Specify Technology Platform**: Choose platform (e.g., "AWS S3", "Azure Blob Storage")
3. **Define Compliance Requirements**: Specify regulations (e.g., "HIPAA", "SOX compliance")
4. **Click Generate**: Wait 1-2 minutes for complete guide
5. **Review & Download**: Read guide in interface, optionally download markdown file

### **Example Inputs:**
- **Healthcare**: Goal: "Patient data backup system" | Platform: "Azure Blob Storage" | Compliance: "HIPAA"
- **Financial**: Goal: "10-year records retention" | Platform: "AWS S3" | Compliance: "SOX compliance"
- **E-commerce**: Goal: "Transaction data archival" | Platform: "Google Cloud Storage" | Compliance: "PCI-DSS"

---

## 🔧 Technical Architecture

### **Agent Workflow**
```
User Input → 3 Parallel Expert Searches → Combined Analysis → Implementation Guide
     ↓              ↓                          ↓                     ↓
Goal/Platform/  Technical Agent         Learning Agent        Step-by-step
Compliance   → Compliance Agent    →   Synthesizes all   →     Guide with
             → Pricing Agent            expert insights       Download Option
```

### **Key Components**
- **Guardrails System**: Input validation, rate limiting, content filtering
- **Pydantic Models**: Type-safe data structures for implementation guides
- **Gradio Interface**: Web UI with immediate display and optional download
- **Specialized Agents**: Platform and compliance-specific expertise
- **Content Formatting**: Display-optimized and download-ready formats

### **Supported Platforms & Compliance**
- **Platforms**: AWS, Azure, Google Cloud, Oracle Cloud, on-premises solutions
- **Compliance**: SOX, GDPR, HIPAA, PCI-DSS, ISO 27001, NIST frameworks

---

## 📊 System Performance

- **Generation Time**: 1-2 minutes per complete guide
- **Guide Length**: 5-8 implementation steps, 2000-4000 words
- **Accuracy**: Based on current official documentation and compliance guides
- **Update Frequency**: Real-time search ensures current information
- **Success Rate**: >95% successful guide generation for valid technical inputs
- **Rate Limiting**: 5 requests per hour per user to manage costs

### **Quick Test Example**
- Goal: `data storage implementation`
- Platform: `AWS S3`
- Compliance: `SOX compliance`

---

**🚀 Execute all cells below to start generating production-ready IT implementation guides!**

In [60]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
import tempfile
from typing import Dict, List
import gradio as gr
import time
import re
import traceback
from datetime import datetime, timedelta

load_dotenv(override=True)

True

In [61]:
# ====== GUARDRAILS CLASS ======
class ImplementationGuardrails:
    def __init__(self):
        self.user_requests = {}  # Track requests per user/session
        self.blocked_keywords = [
            "hack", "exploit", "illegal", "drugs", "violence", "weapon",
            "bomb", "kill", "harm", "suicide", "self-harm"
        ]
        self.max_requests_per_hour = 5
        self.max_topic_length = 300
        self.min_topic_length = 10
        
    def validate_input(self, implementation_goal, technology_platform, compliance_requirements, user_id=None):
        """Validate user input with multiple guardrails"""
        
        print(f"DEBUG: Validating inputs...")
        print(f"DEBUG: Goal length: {len(implementation_goal) if implementation_goal else 0}")
        print(f"DEBUG: Platform: '{technology_platform}'")
        print(f"DEBUG: Compliance: '{compliance_requirements}'")
        
        # 1. Basic input validation for implementation goal
        if not implementation_goal or not implementation_goal.strip():
            return False, "Please enter an implementation goal!"
            
        implementation_goal = implementation_goal.strip()
        
        # 2. Basic validation for technology platform
        if not technology_platform or not technology_platform.strip():
            return False, "Please specify a technology platform!"
            
        # 3. Basic validation for compliance requirements
        if not compliance_requirements or not compliance_requirements.strip():
            return False, "Please specify compliance requirements!"
        
        # 4. Length limits for implementation goal
        if len(implementation_goal) < self.min_topic_length:
            return False, f"Implementation goal must be at least {self.min_topic_length} characters long!"
            
        if len(implementation_goal) > self.max_topic_length:
            return False, f"Implementation goal must be less than {self.max_topic_length} characters!"
        
        # 5. Content filtering
        for text in [implementation_goal, technology_platform, compliance_requirements]:
            text_lower = text.lower()
            for keyword in self.blocked_keywords:
                if keyword in text_lower:
                    return False, "This content is not suitable for implementation guidance!"
        
        # 6. Technical implementation topic check
        if not self._is_technical_topic(implementation_goal, technology_platform):
            return False, "Please enter a topic suitable for IT implementation!"
        
        # 7. Rate limiting (if user_id provided)
        if user_id and not self._check_rate_limit(user_id):
            return False, "You've reached the hourly limit. Please try again later!"
        
        print("DEBUG: Validation passed")
        return True, "Valid input!"
    
    def _is_technical_topic(self, implementation_goal, technology_platform):
        """Check if topic seems like a technical implementation project"""
        combined_text = f"{implementation_goal} {technology_platform}".lower()
        
        # Must be substantial 
        if len(implementation_goal.split()) < 3:
            return False
        
        # Check for technical/IT implementation keywords
        technical_indicators = [
            "aws", "azure", "google cloud", "s3", "blob", "storage", "cloud", "configuration", "setup", "implementation", 
            "infrastructure", "security", "compliance", "data", "records", "backup", "database", "server", "network", 
            "system", "platform", "architecture", "deployment", "monitoring", "optimization", "automation", "devops",
            "bucket", "api", "ssl", "encryption", "authentication", "authorization", "audit", "log", "policy", 
            "access", "management", "enterprise", "business", "container", "kubernetes", "docker", "terraform",
            "cloudformation", "ansible", "jenkins", "gitlab", "github", "ci/cd", "migration", "integration"
        ]
        
        # Check for implementation action words
        action_words = [
            "configure", "setup", "implement", "deploy", "install", "create", "establish", "build", "design", 
            "migrate", "integrate", "automate", "provision", "orchestrate", "scale", "optimize", "secure"
        ]
        
        # Check for cloud platforms
        platform_indicators = [
            "aws", "amazon", "azure", "microsoft", "google", "gcp", "oracle", "ibm", "alibaba", "digitalocean",
            "linode", "vultr", "cloudflare", "fastly", "netlify", "vercel", "heroku"
        ]
        
        has_technical_context = any(indicator in combined_text for indicator in technical_indicators)
        has_action_context = any(action in combined_text for action in action_words)
        has_platform_context = any(platform in combined_text for platform in platform_indicators)
        
        return has_technical_context or has_action_context or has_platform_context
    
    def _check_rate_limit(self, user_id):
        """Simple rate limiting per user"""
        now = datetime.now()
        hour_ago = now - timedelta(hours=1)
        
        if user_id not in self.user_requests:
            self.user_requests[user_id] = []
        
        # Clean old requests
        self.user_requests[user_id] = [
            req_time for req_time in self.user_requests[user_id] 
            if req_time > hour_ago
        ]
        
        # Check limit
        if len(self.user_requests[user_id]) >= self.max_requests_per_hour:
            return False
        
        # Record this request
        self.user_requests[user_id].append(now)
        return True

# Initialize guardrails
guardrails = ImplementationGuardrails()

In [62]:
# ====== PYDANTIC MODELS FOR IMPLEMENTATION GUIDES ======
class ImplementationStep(BaseModel):
    step_number: int = Field(description="Sequential step number")
    title: str = Field(description="Clear, action-oriented step title")
    description: str = Field(description="Detailed implementation instructions")
    technical_details: str = Field(description="Specific configurations, commands, or code")
    verification: str = Field(description="How to verify this step was completed successfully")
    estimated_time: str = Field(description="Time to complete this step")
    prerequisites: List[str] = Field(description="What must be completed before this step")
    troubleshooting: str = Field(description="Common issues and solutions for this step")

class PilotProjectGuide(BaseModel):
    project_title: str = Field(description="Title of the pilot project")
    executive_summary: str = Field(description="2-3 sentence overview for stakeholders")
    project_scope: str = Field(description="What this pilot will accomplish")
    success_criteria: List[str] = Field(description="How to measure pilot success")
    implementation_steps: List[ImplementationStep] = Field(description="Detailed step-by-step implementation")
    rollback_plan: str = Field(description="How to undo changes if needed")
    next_phase_recommendations: List[str] = Field(description="Recommendations for full deployment")
    total_estimated_time: str = Field(description="Total time for pilot implementation")
    budget_considerations: str = Field(description="Cost estimates and budget planning")

# Test Pydantic Model
def test_pydantic_model():
    """Test if the Pydantic model works correctly"""
    try:
        test_step = ImplementationStep(
            step_number=1,
            title="Test Step",
            description="Test description",
            technical_details="Test technical details",
            verification="Test verification",
            estimated_time="30 minutes",
            prerequisites=["Test prerequisite"],
            troubleshooting="Test troubleshooting"
        )
        
        test_guide = PilotProjectGuide(
            project_title="Test Project",
            executive_summary="Test summary",
            project_scope="Test scope",
            success_criteria=["Test criteria"],
            implementation_steps=[test_step],
            rollback_plan="Test rollback",
            next_phase_recommendations=["Test recommendation"],
            total_estimated_time="2 hours",
            budget_considerations="Test budget"
        )
        
        print("DEBUG: Pydantic model test PASSED")
        return True
    except Exception as e:
        print(f"DEBUG: Pydantic model test FAILED: {e}")
        return False

# Run Pydantic test
test_pydantic_model()


DEBUG: Pydantic model test PASSED


True

In [64]:
# ====== SPECIALIZED AGENT DEFINITIONS ======

# Technical Instructions Agent - NOW PLATFORM FLEXIBLE
TECHNICAL_INSTRUCTIONS = "You are a cloud infrastructure technical expert specializing in implementation guidance for IT administrators. \
Given a search term that specifies a cloud platform and use case, you search for current official documentation, configuration guides, \
and step-by-step implementation instructions for that specific platform. Focus on: specific configurations, access policies, \
lifecycle rules, storage classes, and monitoring setup for the specified cloud service. Prioritize official documentation from \
the platform vendor (AWS, Microsoft Azure, Google Cloud, etc.) and current best practices. Target IT administrators who need to \
actually implement and configure cloud services for business use. Include code examples, configuration snippets, and troubleshooting tips. \
Summary must be 2-3 paragraphs and less than 300 words with actionable technical details."

technical_agent = Agent(
    name="Multi-Platform Technical Expert",
    instructions=TECHNICAL_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

# Compliance Agent - NOW COMPLIANCE-TYPE FLEXIBLE
COMPLIANCE_INSTRUCTIONS = "You are a compliance specialist focused on data retention and regulatory requirements. \
Given a search term that specifies compliance frameworks and data types, you search for regulatory requirements, \
compliance frameworks, and legal obligations related to cloud storage under those specific regulations. \
Focus on the specific compliance standard mentioned (SOX, GDPR, HIPAA, PCI-DSS, ISO 27001, etc.) and their requirements for \
cloud data storage, retention, access controls, and audit trails. Prioritize official regulatory sources and \
industry compliance guides specific to the mentioned framework. Target IT administrators who must ensure legal \
compliance with the specified regulations. Include specific requirements, timeframes, and mandatory controls. \
Summary must be 2-3 paragraphs and less than 300 words with compliance-focused details."

compliance_agent = Agent(
    name="Multi-Framework Compliance Expert",
    instructions=COMPLIANCE_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

# Pricing Agent - NOW PLATFORM FLEXIBLE
PRICING_INSTRUCTIONS = "You are a cloud cost optimization specialist focused on storage pricing for business use cases. \
Given a search term that specifies a cloud platform and storage requirements, you search for current pricing, \
cost calculators, and cost optimization strategies for that specific platform. Focus on storage class pricing, \
lifecycle cost optimization, data transfer costs, and total cost of ownership for the specified cloud service. \
Prioritize official pricing pages and cost calculators from the platform vendor (AWS, Azure, Google Cloud, etc.). \
Target IT administrators who need to budget and optimize storage costs. Include specific pricing tiers, cost-saving recommendations, \
and budget planning guidance. Summary must be 2-3 paragraphs and less than 300 words with actionable cost insights."

pricing_agent = Agent(
    name="Multi-Platform Cost Optimization Expert",
    instructions=PRICING_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

# Implementation Guide Agent
def get_implementation_instructions():
    return (
        "You are a senior IT consultant specializing in creating practical pilot project implementation guides for any cloud platform. "
        "You are receiving expert research from 3 specialists: cloud platform technical expert, compliance specialist, "
        "and cost optimization expert. Create a comprehensive, step-by-step pilot project implementation guide "
        "that IT professionals can follow to implement the solution using the specified technology platform and compliance framework. "
        "Focus on: actionable steps, platform-specific configurations, verification procedures, troubleshooting guidance, "
        "risk mitigation, and rollback procedures. Include technical details like commands, policies, and configurations "
        "specific to the mentioned cloud platform. Target experienced IT administrators who need to demonstrate value "
        "through a successful pilot project. Make it production-ready but scoped as a proof-of-concept pilot. "
        "Create 5-8 detailed implementation steps that are specific to the technology platform mentioned."
    )

def create_implementation_agent():
    return Agent(
        name="Multi-Platform Implementation Guide Specialist",
        instructions=get_implementation_instructions(),
        model="gpt-4o-mini",
        output_type=PilotProjectGuide,
    )


In [65]:
# ====== SPECIALIZED SEARCH WORKFLOW FUNCTIONS ======

async def technical_search(implementation_goal: str, technology_platform: str):
    """Use technical agent to search for platform-specific implementation guidance"""
    print(f"DEBUG: Starting technical search for {technology_platform}")
    try:
        technical_query = f"{technology_platform} configuration implementation for {implementation_goal}"
        print(f"DEBUG: Technical query: {technical_query}")
        result = await Runner.run(technical_agent, technical_query)
        print(f"DEBUG: Technical search completed successfully")
        return result.final_output
    except Exception as e:
        print(f"DEBUG: Technical search failed: {e}")
        raise

async def compliance_search(implementation_goal: str, compliance_requirements: str):
    """Use compliance agent to search for specific regulatory requirements"""
    print(f"DEBUG: Starting compliance search for {compliance_requirements}")
    try:
        compliance_query = f"{compliance_requirements} compliance regulations cloud storage for {implementation_goal}"
        print(f"DEBUG: Compliance query: {compliance_query}")
        result = await Runner.run(compliance_agent, compliance_query)
        print(f"DEBUG: Compliance search completed successfully")
        return result.final_output
    except Exception as e:
        print(f"DEBUG: Compliance search failed: {e}")
        raise

async def pricing_search(implementation_goal: str, technology_platform: str):
    """Use pricing agent to search for platform-specific cost analysis"""
    print(f"DEBUG: Starting pricing search for {technology_platform}")
    try:
        pricing_query = f"{technology_platform} pricing cost optimization for {implementation_goal}"
        print(f"DEBUG: Pricing query: {pricing_query}")
        result = await Runner.run(pricing_agent, pricing_query)
        print(f"DEBUG: Pricing search completed successfully")
        return result.final_output
    except Exception as e:
        print(f"DEBUG: Pricing search failed: {e}")
        raise

async def perform_specialized_searches(implementation_goal: str, technology_platform: str, compliance_requirements: str):
    """Orchestrate all 3 specialized agents with specific topics"""
    print("DEBUG: Starting specialized agent research...")
    
    try:
        # Run all 3 agents in parallel with specific topics
        tasks = [
            asyncio.create_task(technical_search(implementation_goal, technology_platform)),
            asyncio.create_task(compliance_search(implementation_goal, compliance_requirements)),
            asyncio.create_task(pricing_search(implementation_goal, technology_platform))
        ]
        
        results = await asyncio.gather(*tasks)
        
        print("DEBUG: Finished specialized research successfully")
        return {
            "technical_guidance": results[0],
            "compliance_requirements": results[1], 
            "cost_analysis": results[2]
        }
    except Exception as e:
        print(f"DEBUG: Specialized searches failed: {e}")
        raise

async def generate_implementation_guide(implementation_goal, technology_platform, compliance_requirements):
    """Generate implementation guide with specific platform and compliance requirements"""
    try:
        print(f"DEBUG: Creating implementation guide for: {implementation_goal}")
        print(f"DEBUG: Platform: {technology_platform}")
        print(f"DEBUG: Compliance: {compliance_requirements}")
        
        # Get specialized research from 3 expert agents with specific topics
        print("DEBUG: Step 1: Starting specialized searches...")
        specialized_results = await perform_specialized_searches(
            implementation_goal, technology_platform, compliance_requirements
        )
        print("DEBUG: Step 1: Specialized searches completed successfully")
        
        # Create implementation guide with specialized expert knowledge
        print("DEBUG: Step 2: Creating implementation agent...")
        implementation_agent = create_implementation_agent()
        print("DEBUG: Step 2: Implementation agent created successfully")
        
        # Format for pilot project context with specific details
        expert_input = f"""
Create a pilot project implementation guide for: {implementation_goal}
Technology Platform: {technology_platform}
Compliance Requirements: {compliance_requirements}

TECHNICAL IMPLEMENTATION GUIDANCE FOR {technology_platform.upper()}:
{specialized_results['technical_guidance']}

COMPLIANCE REQUIREMENTS FOR {compliance_requirements.upper()}:
{specialized_results['compliance_requirements']}

COST ANALYSIS FOR {technology_platform.upper()}:
{specialized_results['cost_analysis']}

Create a comprehensive step-by-step pilot project guide that demonstrates value while minimizing risk.
Focus on {technology_platform} specific configurations and {compliance_requirements} specific requirements.
Ensure all steps are tailored to the {technology_platform} platform and meet {compliance_requirements} standards.
"""
        
        print("DEBUG: Step 3: Starting implementation guide generation...")
        print(f"DEBUG: Input length: {len(expert_input)} characters")
        
        with trace("Custom Implementation Guide Creation"):
            result = await Runner.run(implementation_agent, expert_input)
            print("DEBUG: Step 3: Runner.run completed")
            
            # Debug the result
            print(f"DEBUG: Result type: {type(result)}")
            print(f"DEBUG: Result has final_output: {hasattr(result, 'final_output')}")
            
            if hasattr(result, 'final_output'):
                implementation_guide = result.final_output
                print(f"DEBUG: Final output type: {type(implementation_guide)}")
                
                # Check if it's the right type
                if isinstance(implementation_guide, PilotProjectGuide):
                    print(f"DEBUG: Implementation guide is correct type")
                    print(f"DEBUG: Project title: {implementation_guide.project_title}")
                    print(f"DEBUG: Number of steps: {len(implementation_guide.implementation_steps)}")
                else:
                    print(f"DEBUG: WARNING - Implementation guide is wrong type: {type(implementation_guide)}")
                    print(f"DEBUG: Content: {str(implementation_guide)[:500]}...")
            else:
                print("DEBUG: ERROR: No final_output in result")
                return None, "No final_output in agent result"
        
        print("DEBUG: Custom implementation guide created successfully!")
        return implementation_guide, None
        
    except Exception as e:
        error_msg = f"Error generating guide: {str(e)}"
        print(f"DEBUG: FULL ERROR: {error_msg}")
        print(f"DEBUG: TRACEBACK: {traceback.format_exc()}")
        return None, error_msg


In [66]:
def format_implementation_guide_for_download(guide: PilotProjectGuide, technology_platform: str, compliance_requirements: str):
    """Format the implementation guide as downloadable markdown"""
    
    try:
        print("DEBUG: Starting download formatting")
        
        markdown_content = f"""# {guide.project_title}

**Technology Platform:** {technology_platform}  
**Compliance Framework:** {compliance_requirements}  
**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Executive Summary
{guide.executive_summary}

## Project Scope
{guide.project_scope}

## Success Criteria
"""
        
        for i, criteria in enumerate(guide.success_criteria, 1):
            markdown_content += f"{i}. {criteria}\n"
        
        markdown_content += f"""
## Budget Considerations
{guide.budget_considerations}

## Total Estimated Time
{guide.total_estimated_time}

---

## Implementation Steps

"""
        
        for step in guide.implementation_steps:
            markdown_content += f"""
### Step {step.step_number}: {step.title}

**Estimated Time:** {step.estimated_time}

**Prerequisites:**
"""
            for prereq in step.prerequisites:
                markdown_content += f"- {prereq}\n"
            
            markdown_content += f"""

**Description:**
{step.description}

**Technical Details:**
```
{step.technical_details}
```

**Verification:**
{step.verification}

**Troubleshooting:**
{step.troubleshooting}

---
"""
        
        markdown_content += f"""
## Rollback Plan
{guide.rollback_plan}

## Next Phase Recommendations
"""
        
        for i, rec in enumerate(guide.next_phase_recommendations, 1):
            markdown_content += f"{i}. {rec}\n"
        
        markdown_content += f"""

---
*Multi-Platform Implementation guide generated by AI Expert System*  
*Platform: {technology_platform}*  
*Compliance: {compliance_requirements}*  
*Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*
"""
        
        print("DEBUG: Download formatting completed successfully")
        return markdown_content
        
    except Exception as e:
        print(f"DEBUG: Error in download formatting: {e}")
        print(f"DEBUG: Download formatting traceback: {traceback.format_exc()}")
        raise

def format_implementation_guide_for_display(guide: PilotProjectGuide, technology_platform: str, compliance_requirements: str):
    """Format the implementation guide for web display"""
    
    try:
        print("DEBUG: Starting display formatting")
        
        content = f"""# {guide.project_title}

**🔧 Technology Platform:** {technology_platform}  
**⚖️ Compliance Framework:** {compliance_requirements}

## 📋 Executive Summary
{guide.executive_summary}

## 🎯 Project Scope  
{guide.project_scope}

## ✅ Success Criteria
"""
        
        for i, criteria in enumerate(guide.success_criteria, 1):
            content += f"{i}. {criteria}\n"
        
        content += f"""

## 💰 Budget Considerations
{guide.budget_considerations}

## ⏱️ Total Estimated Time
{guide.total_estimated_time}

---

## 🔧 Implementation Steps
"""
        
        for step in guide.implementation_steps:
            content += f"""

### Step {step.step_number}: {step.title}

**⏱️ Time:** {step.estimated_time}

**📋 Prerequisites:**
"""
            for prereq in step.prerequisites:
                content += f"- {prereq}\n"
            
            content += f"""

**📝 Description:**
{step.description}

**🔧 Technical Details:**
```
{step.technical_details}
```

**✅ Verification:**
{step.verification}

**🚨 Troubleshooting:**
{step.troubleshooting}

---
"""
        
        content += f"""

## 🔄 Rollback Plan
{guide.rollback_plan}

## 🚀 Next Phase Recommendations
"""
        
        for i, rec in enumerate(guide.next_phase_recommendations, 1):
            content += f"{i}. {rec}\n"
        
        print("DEBUG: Display formatting completed successfully")
        return content
        
    except Exception as e:
        print(f"DEBUG: Error in display formatting: {e}")
        print(f"DEBUG: Display formatting traceback: {traceback.format_exc()}")
        raise


In [67]:
# ====== GRADIO INTERFACE ======

def create_implementation_interface():
    """Create the implementation guide interface with separate display and download"""
    
    current_implementation_guide = None
    current_download_content = None
    current_filename = None
    
    def generate_guide(implementation_goal, technology_platform, compliance_requirements, request: gr.Request):
        nonlocal current_implementation_guide, current_download_content, current_filename
        
        print(f"DEBUG: Starting generate_guide")
        print(f"DEBUG: Goal: '{implementation_goal}'")
        print(f"DEBUG: Platform: '{technology_platform}'")
        print(f"DEBUG: Compliance: '{compliance_requirements}'")
        
        # Validate all inputs
        user_id = request.client.host if request and request.client else None
        is_valid, message = guardrails.validate_input(
            implementation_goal, technology_platform, compliance_requirements, user_id
        )
        
        if not is_valid:
            print(f"DEBUG: Validation failed: {message}")
            return f"❌ {message}", "*Your implementation guide will appear here after generation...*", gr.update(visible=False)
        
        print("DEBUG: Validation passed")
        
        try:
            print("DEBUG: Creating event loop")
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            
            print("DEBUG: Running generate_implementation_guide")
            implementation_guide, error = loop.run_until_complete(
                generate_implementation_guide(implementation_goal, technology_platform, compliance_requirements)
            )
            loop.close()
            print("DEBUG: Event loop completed")
            
            if error:
                print(f"DEBUG: Error returned: {error}")
                return f"❌ {error}", "*Your implementation guide will appear here after generation...*", gr.update(visible=False)
            
            if implementation_guide:
                print("DEBUG: Implementation guide generated, starting formatting")
                current_implementation_guide = implementation_guide
                
                try:
                    # Format for display
                    formatted_content = format_implementation_guide_for_display(
                        implementation_guide, technology_platform, compliance_requirements
                    )
                    print("DEBUG: Display formatting completed")
                    
                    # Prepare download content
                    current_download_content = format_implementation_guide_for_download(
                        implementation_guide, technology_platform, compliance_requirements
                    )
                    print("DEBUG: Download formatting completed")
                    
                    # Create filename
                    safe_title = re.sub(r'[^a-zA-Z0-9\s-]', '', implementation_guide.project_title)
                    safe_title = re.sub(r'\s+', '_', safe_title)
                    safe_platform = re.sub(r'[^a-zA-Z0-9\s-]', '', technology_platform)
                    safe_platform = re.sub(r'\s+', '_', safe_platform)
                    current_filename = f"{safe_title}_{safe_platform}_Implementation_Guide.md"
                    print(f"DEBUG: Filename created: {current_filename}")
                    
                    print("DEBUG: Returning success response")
                    return (f"✅ Implementation guide generated successfully! Combined insights for {technology_platform} implementation with {compliance_requirements} compliance.", 
                           formatted_content, 
                           gr.update(visible=True))  # Show download button
                           
                except Exception as format_error:
                    print(f"DEBUG: Error in formatting: {format_error}")
                    print(f"DEBUG: Formatting traceback: {traceback.format_exc()}")
                    return (f"❌ Error formatting content: {str(format_error)}", 
                           "*Your implementation guide will appear here after generation...*", 
                           gr.update(visible=False))
            else:
                print("DEBUG: No implementation guide returned")
                return ("❌ Failed to generate implementation guide", 
                       "*Your implementation guide will appear here after generation...*", 
                       gr.update(visible=False))
                
        except Exception as e:
            print(f"DEBUG: Main exception: {str(e)}")
            print(f"DEBUG: Main traceback: {traceback.format_exc()}")
            return (f"❌ Error: {str(e)}", 
                   "*Your implementation guide will appear here after generation...*", 
                   gr.update(visible=False))
    
    def download_guide():
        """Handle the download when user clicks download button"""
        nonlocal current_download_content, current_filename
    
        if current_download_content and current_filename:
            try:
            # Use tempfile for Hugging Face Spaces
                with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False, encoding='utf-8') as temp_file:
                    temp_file.write(current_download_content)
                    return temp_file.name
            except Exception as e:
                print(f"Download error: {e}")
                return None
        else:
            print("DEBUG: No content available for download")
            return None
    
    with gr.Blocks(title="Multi-Platform IT Implementation Guide Generator", theme=gr.themes.Soft()) as app:
        gr.Markdown("# 🛠️ Multi-Platform IT Implementation Guide Generator")
        gr.Markdown("**Platform-Agnostic Expert System:** Generate comprehensive implementation guides for any cloud platform with any compliance requirements.")
        
        # Usage guidelines - ALWAYS VISIBLE
        gr.Markdown("""
        ### 📋 Usage Guidelines
        - **Any cloud platform** - AWS, Azure, Google Cloud, on-premises solutions
        - **Any compliance framework** - SOX, GDPR, HIPAA, PCI-DSS, ISO 27001, etc.
        - **Pilot project focus** - Risk-managed, proof-of-concept approach  
        - **Production-ready guidance** - Real configurations and procedures
        - **5 requests per hour limit** - To manage system costs
        """)
        
        # Input section with 3 specific fields
        with gr.Row():
            implementation_goal = gr.Textbox(
                label="📋 Implementation Goal", 
                placeholder="e.g., Long-term storage of accounting records for 10 years with automated lifecycle management",
                lines=2,
                scale=2
            )
        
        with gr.Row():
            technology_platform = gr.Textbox(
                label="🔧 Technology Platform", 
                placeholder="e.g., AWS S3, Azure Blob Storage, Google Cloud Storage",
                lines=1,
                scale=1
            )
            
            compliance_requirements = gr.Textbox(
                label="⚖️ Compliance Requirements", 
                placeholder="e.g., SOX compliance, GDPR, HIPAA, PCI-DSS",
                lines=1,
                scale=1
            )
        
        # Examples section
        with gr.Accordion("💡 Example Combinations", open=False):
            gr.Markdown("""
            **Simple Test:**
            - Goal: "data storage implementation"
            - Platform: "AWS S3"
            - Compliance: "SOX compliance"
            
            **Healthcare Example:** 
            - Goal: "Medical records storage and patient data backup with audit trails"
            - Platform: "Azure Blob Storage" 
            - Compliance: "HIPAA compliance"
            
            **Financial Services:**
            - Goal: "Financial reporting data warehouse with audit trails and automated retention"
            - Platform: "AWS S3"
            - Compliance: "SOX compliance"
            """)
        
        generate_btn = gr.Button("🚀 Generate Implementation Guide", variant="primary", size="lg")
        
        # Status section
        status_output = gr.Textbox(label="Status", lines=2)
        
        # Main content display - ALWAYS VISIBLE
        gr.Markdown("## 📖 Implementation Guide")
        content_display = gr.Markdown(
            value="*Your implementation guide will appear here after generation...*",
            elem_classes=["implementation-guide"]
        )
        
        # Download section - Initially hidden
        with gr.Row(visible=False) as download_section:
            gr.Markdown("## 📥 Download Guide")
            gr.Markdown("Click below to download the complete implementation guide as a Markdown file.")
            
            with gr.Row():
                download_btn = gr.Button("📄 Download Implementation Guide", variant="secondary", size="lg")
                download_file = gr.File(
                    label="Download will appear here",
                    visible=True,
                    interactive=False
                )
        
        # Connect the generate button
        generate_btn.click(
            generate_guide,
            inputs=[implementation_goal, technology_platform, compliance_requirements],
            outputs=[status_output, content_display, download_section]
        )
        
        # Connect the download button
        download_btn.click(
            download_guide,
            outputs=[download_file]
        )
    
    return app

In [68]:
# ====== LAUNCH THE APPLICATION ======

# For Jupyter Notebook
app = create_implementation_interface()
app.launch(
    share=False,  # Spaces handles sharing
    server_name="0.0.0.0",  # Required for Spaces
    server_port=7860,  # Default port
    show_error=True,
    favicon_path=None  # Optional: add favicon
)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://dba9b87f3383d1cf6a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


DEBUG: Starting generate_guide
DEBUG: Goal: 'data storage implementation'
DEBUG: Platform: 'AWS S3'
DEBUG: Compliance: 'SOX compliance'
DEBUG: Validating inputs...
DEBUG: Goal length: 27
DEBUG: Platform: 'AWS S3'
DEBUG: Compliance: 'SOX compliance'
DEBUG: Validation passed
DEBUG: Validation passed
DEBUG: Creating event loop
DEBUG: Running generate_implementation_guide
DEBUG: Creating implementation guide for: data storage implementation
DEBUG: Platform: AWS S3
DEBUG: Compliance: SOX compliance
DEBUG: Step 1: Starting specialized searches...
DEBUG: Starting specialized agent research...
DEBUG: Starting technical search for AWS S3
DEBUG: Technical query: AWS S3 configuration implementation for data storage implementation
DEBUG: Starting compliance search for SOX compliance
DEBUG: Compliance query: SOX compliance compliance regulations cloud storage for data storage implementation
DEBUG: Starting pricing search for AWS S3
DEBUG: Pricing query: AWS S3 pricing cost optimization for data sto